In [7]:
import cv2
import time
import torch
import numpy as np
from skimage.transform import resize

In [13]:
def get_blocc(arr):
    blocc = np.array([resize(frame/255, output_shape=(112, 200), preserve_range=True) for frame in arr])

    blocc = blocc[:, :, 44:44+112, :]
    blocc = blocc.transpose(3, 0, 1, 2)  # ch, fr, h, w
    blocc = np.expand_dims(blocc, axis=0)  # batch axis
    blocc = np.float32(blocc)
    blocc = torch.from_numpy(blocc)
    return blocc

In [19]:
def predict(video, model):
    cap = cv2.VideoCapture(video) 
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 0.5
    fontColor              = (255,0,0)
    lineType               = 2
    frames = 0
    
    # Read until video is completed
    start_time = time.time()
    i = 0
    arr = []
    score = 0
    while(cap.isOpened()):
      # Capture frame-by-frame
        ret, frame = cap.read()
        frames += 1
        end_time = time.time()-start_time
        frame_rate = frames/end_time

        if(i<16):
            arr.append(frame)
            i+=1
        else:
            i = 0
            arr = []

        if (ret == True):
            if(len(arr) == 16):
                X = get_blocc(arr)
                prediction = model(X)
                score = prediction.data.cpu().numpy()
                score = str(score)
            # Display the resulting frame
            message = "framerate = {}, frame_id = {}".format(score, i)
            cv2.putText(frame, message, 
                        bottomLeftCornerOfText, 
                        font, 
                        fontScale,
                        fontColor,
                        lineType)
            cv2.imshow('Frame',frame)

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            # Break the loop
        else: 
            break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [20]:
predict(video='/home/nevin/nevin/datasets/anomaly detection/assault/Assault015_x264.mp4')

/home/nevin/anaconda3/envs/pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/nevin/anaconda3/envs/pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
